# QC-rapport automatiseren in Excel met Python & openpyxl

**Use case (ingenieurs):** meetdata controleren t.o.v. specificatiegrenzen, 
overtredingen markeren, statistieken berekenen en automatisch een rapportblad maken.

*Laatste update:* 2026-02-27 22:28


## Context en leerdoelen
In industriële omgevingen worden metingen (bv. temperatuur, diameter, trillingen) vaak in Excel
gelogd. Deze notebook demonstreert hoe je met **openpyxl** een reproduceerbare QC-pipeline bouwt:

- Overschrijdingen (out-of-spec) **visueel markeren** in het gegevensblad.
- **Statistieken** berekenen (gemiddelde, minimum, maximum, standaardafwijking).
- Een **rapportblad** genereren met een QC-conclusie.
- (Optioneel) Een **grafiek** genereren en in het Excel-bestand inbedden.

> **Opmerking:** Deze notebook verwacht een Excelbestand met kolommen: `Timestamp`, `Sensorwaarde`, `Ondergrens`, `Bovengrens` in een werkblad, bv. `Batch_01`.


## Vereisten
- Python 3
- Pakketten: `openpyxl`, `matplotlib` (alleen voor de optionele grafiek), `pillow` (wordt door openpyxl gebruikt voor het inbedden van afbeeldingen).

Installeer zo nodig lokaal via:
```bash
pip install openpyxl matplotlib pillow
```


In [ ]:
# 📦 Imports en configuratie
from pathlib import Path
from statistics import mean, stdev
from datetime import datetime
import math

from openpyxl import load_workbook, Workbook
from openpyxl.styles import PatternFill, Font
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image as XLImage

# (Optioneel) voor grafieken
import matplotlib.pyplot as plt

# Bestandsnamen en instellingen
INPUT_FILE = 'meetdata.xlsx'      # verwacht inputbestand
INPUT_SHEET = 'Batch_01'         # naam van het werkblad met data
OUTPUT_FILE = 'QC_rapport_batch01.xlsx'
RAPPORT_SHEET = 'Rapport'

# Kleuren
FILL_OOS = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')  # lichtrood
FILL_HEADER = PatternFill(start_color='FFEE93', end_color='FFEE93', fill_type='solid')  # zacht geel

print('Configuratie klaar. Werkmap:', Path.cwd())


## (Optioneel) Genereer voorbeeld-Excel voor demo
Als je nog geen `meetdata.xlsx` hebt, voer dan onderstaand blok uit om een
synthetische dataset te maken. **Gebruik alleen voor demo/doeleinden**.


In [ ]:
# ⚠️ Alleen uitvoeren als je nog geen meetdata.xlsx hebt
import random

def genereer_voorbeeld_excel(path='meetdata.xlsx', sheet='Batch_01', n=200,
                               low=10.0, high=13.0, mu=11.5, sigma=0.7, seed=42):
    random.seed(seed)
    wb = Workbook()
    ws = wb.active
    ws.title = sheet
    ws.append(['Timestamp', 'Sensorwaarde', 'Ondergrens', 'Bovengrens'])
    t0 = datetime(2026, 2, 27, 8, 0, 0)
    for i in range(n):
        ts = t0.replace(second=(t0.second + i) % 60)
        val = random.gauss(mu, sigma)
        # injecteer af en toe een overschrijding
        if i % 47 == 0:
            val = high + random.uniform(0.2, 1.2)
        elif i % 73 == 0:
            val = low - random.uniform(0.2, 1.0)
        ws.append([ts.strftime('%H:%M:%S'), round(val, 3), low, high])

    # eenvoudige kolombreedtes
    for col, width in zip(['A','B','C','D'], [12, 14, 12, 12]):
        ws.column_dimensions[col].width = width

    wb.save(path)
    print(f'Voorbeeldbestand geschreven: {path}')

# genereer_voorbeeld_excel()  # ← decommentarieer om te maken


## Stap 1 – Inlezen en basale validatie
We lezen het werkblad in en vormen een lijst van metingen.
We doen ook een elementaire type- en null-check.


In [ ]:
def lees_metingen(pad, sheetnaam):
    wb = load_workbook(pad)
    if sheetnaam not in wb.sheetnames:
        raise ValueError(f'Werkblad {sheetnaam} niet gevonden. Beschikbaar: {wb.sheetnames}')
    ws = wb[sheetnaam]

    records = []
    for r in ws.iter_rows(min_row=2, values_only=True):
        timestamp, value, low, high = r
        if value is None or low is None or high is None:
            # sla onvolledige rijen over
            continue
        try:
            v = float(value)
            lo = float(low)
            hi = float(high)
        except Exception:
            # sla rijen met niet-numerieke waarden over
            continue
        records.append({
            'timestamp': timestamp,
            'value': v,
            'low': lo,
            'high': hi
        })
    return wb, ws, records

wb, ws, metingen = lees_metingen(INPUT_FILE, INPUT_SHEET)
print(f'Aantal geldige metingen: {len(metingen)}')
print('Eerste 3 records:', metingen[:3])


## Stap 2 – Overschrijdingen detecteren en markeren
We markeren `Sensorwaarde` rood als die buiten `[Ondergrens, Bovengrens]` ligt.


In [ ]:
def markeer_overschrijdingen(ws):
    n_oos = 0
    for row in ws.iter_rows(min_row=2):
        try:
            value = float(row[1].value)
            low = float(row[2].value)
            high = float(row[3].value)
        except Exception:
            continue
        if value < low or value > high:
            row[1].fill = FILL_OOS
            n_oos += 1
    return n_oos

aantal_oos = markeer_overschrijdingen(ws)
print(f'Overschrijdingen gemarkeerd: {aantal_oos}')


## Stap 3 – Statistieken berekenen
We berekenen gemiddelde, min, max en standaardafwijking voor de `Sensorwaarde`.
> Let op: `stdev` vereist minstens 2 waarden.


In [ ]:
def bereken_stats(metingen):
    waarden = [m['value'] for m in metingen]
    if not waarden:
        return {
            'gemiddelde': math.nan,
            'minimum': math.nan,
            'maximum': math.nan,
            'std': math.nan
        }
    gem = mean(waarden)
    minimum = min(waarden)
    maximum = max(waarden)
    std = stdev(waarden) if len(waarden) > 1 else float('nan')
    return {
        'gemiddelde': gem,
        'minimum': minimum,
        'maximum': maximum,
        'std': std
    }

stats = bereken_stats(metingen)
stats


## Stap 4 – Rapportblad genereren
We maken (of overschrijven) een blad `Rapport` met kerncijfers en een QC-conclusie.


In [ ]:
def maak_of_leeg_rapport(wb, naam):
    if naam in wb.sheetnames:
        ws = wb[naam]
        ws.delete_rows(1, ws.max_row)
    else:
        ws = wb.create_sheet(naam)
    return ws

def schrijf_rapport(wb, stats, totaal, oos):
    rapport = maak_of_leeg_rapport(wb, RAPPORT_SHEET)
    header = ['Parameter', 'Waarde']
    rapport.append(header)
    for cell in rapport[1]:
        cell.fill = FILL_HEADER
        cell.font = Font(bold=True)

    rapport.append(['Aantal metingen', totaal])
    rapport.append(['Aantal overschrijdingen', oos])
    pct = (oos / totaal * 100.0) if totaal else 0.0
    rapport.append(['% overschrijdingen', round(pct, 3)])
    rapport.append(['Gemiddelde', stats['gemiddelde']])
    rapport.append(['Minimum', stats['minimum']])
    rapport.append(['Maximum', stats['maximum']])
    rapport.append(['Standaardafwijking', stats['std']])

    conclusie = '✔️ Conform' if oos == 0 else '⚠️ Niet conform'
    rapport.append(['Conclusie', conclusie])
    rapport.append(['Gegenereerd op', datetime.now().strftime('%Y-%m-%d %H:%M:%S')])

    # kolombreedtes
    rapport.column_dimensions['A'].width = 24
    rapport.column_dimensions['B'].width = 18
    return rapport

rapport_ws = schrijf_rapport(wb, stats, totaal=len(metingen), oos=aantal_oos)
'Rapportblad geschreven.'


## Stap 5 – (Optioneel) Grafiek genereren en inbedden
We plotten de tijdreeks van `Sensorwaarde` en markeren overschrijdingen.
De afbeelding wordt onderaan het **Rapport**-blad ingevoegd.


In [ ]:
def genereer_grafiek_png(metingen, pad='qc_plot.png'):
    if not metingen:
        return None
    y = [m['value'] for m in metingen]
    lo = [m['low'] for m in metingen]
    hi = [m['high'] for m in metingen]
    x = list(range(1, len(metingen)+1))

    # detecteer overschrijdingen
    oos_x = [i for i, m in enumerate(metingen, start=1) if (m['value'] < m['low'] or m['value'] > m['high'])]
    oos_y = [y[i-1] for i in oos_x]

    plt.figure(figsize=(8, 3.5))
    plt.plot(x, y, label='Sensorwaarde')
    plt.plot(x, lo, linestyle='--', label='Ondergrens')
    plt.plot(x, hi, linestyle='--', label='Bovengrens')
    if oos_x:
        plt.scatter(oos_x, oos_y, color='red', label='Overschrijding', zorder=3)
    plt.xlabel('Index')
    plt.ylabel('Waarde')
    plt.title('QC-plot: Sensorwaarde t.o.v. grenzen')
    plt.legend()
    plt.tight_layout()
    plt.savefig(pad)
    plt.close()
    return pad

def voeg_grafiek_in_excel(wb, rapport_ws, png_pad):
    if png_pad and Path(png_pad).exists():
        img = XLImage(png_pad)
        # plaatsing: onder de laatste rij van de tabel
        anchor_row = rapport_ws.max_row + 2
        anchor_cell = f'A{anchor_row}'
        rapport_ws.add_image(img, anchor_cell)
        return True
    return False

png = genereer_grafiek_png(metingen)
_ = voeg_grafiek_in_excel(wb, rapport_ws, png)
'Grafiek toegevoegd aan rapport (indien beschikbaar).'


## Stap 6 – Opslaan
We schrijven het resultaat naar `OUTPUT_FILE`.


In [ ]:
wb.save(OUTPUT_FILE)
print(f'Bestand opgeslagen als: {OUTPUT_FILE}')


## (Optioneel) Batchverwerking van meerdere Excelbestanden
Onderstaande helper laat zien hoe je dezelfde pipeline kan toepassen op een map met bestanden.
Pas zo nodig de bestands- en bladnamen aan.


In [ ]:
def verwerk_bestand(input_path, input_sheet, output_path):
    wb, ws, metingen = lees_metingen(input_path, input_sheet)
    oos = markeer_overschrijdingen(ws)
    st = bereken_stats(metingen)
    rapport_ws = schrijf_rapport(wb, st, totaal=len(metingen), oos=oos)
    try:
        png = genereer_grafiek_png(metingen, pad=str(Path(output_path).with_suffix('.png')))
        _ = voeg_grafiek_in_excel(wb, rapport_ws, png)
    except Exception:
        pass
    wb.save(output_path)
    return oos

# Voorbeeld om een hele map te verwerken (uitcommentariëren om te gebruiken):
# for inp in Path('.').glob('*.xlsx'):
#     if inp.name.startswith('~$') or inp.name.startswith('QC_'):
#         continue
#     try:
#         outp = inp.with_name(f'QC_rapport_{inp.stem}.xlsx')
#         oos = verwerk_bestand(inp, INPUT_SHEET, outp)
#         print(f'✔ Verwerkt {inp.name} → {outp.name} (OOS={oos})')
#     except Exception as e:
#         print(f'⚠ Kon {inp.name} niet verwerken:', e)
